<a href="https://colab.research.google.com/github/caiobaptistaa/Extract_ClimateData_ECMWF/blob/main/Shape_to_df_Passo_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2> Agregando o CSV do notebook passado por centróides municipais <h2>

In [ ]:
#pip install geopandas

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from google.colab import drive

drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/Climate_Data_Files/v_wind2017.csv')
#data = data.drop(columns = 'Unnamed: 0',  axis = 1)
pd.pandas.set_option('display.max_columns', None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data.head()

,longitude,latitude,time,v10,geometry
0,-57.400002,-30.299999,2017-04-01 12:00:00,0.079208,POINT (-57.400001525878906 -30.299999237060547)
1,-57.400002,-30.299999,2017-01-01 12:00:00,-0.174589,POINT (-57.400001525878906 -30.299999237060547)
2,-57.400002,-30.299999,2017-02-01 12:00:00,-0.031723,POINT (-57.400001525878906 -30.299999237060547)
3,-57.400002,-30.299999,2017-03-01 12:00:00,0.543439,POINT (-57.400001525878906 -30.299999237060547)
4,-57.400002,-30.299999,2017-05-01 12:00:00,0.700424,POINT (-57.400001525878906 -30.299999237060547)


In [ ]:
data = data.drop(columns = "geometry")
data.head()

,longitude,latitude,time,v10
0,-57.400002,-30.299999,2017-04-01 12:00:00,0.079208
1,-57.400002,-30.299999,2017-01-01 12:00:00,-0.174589
2,-57.400002,-30.299999,2017-02-01 12:00:00,-0.031723
3,-57.400002,-30.299999,2017-03-01 12:00:00,0.543439
4,-57.400002,-30.299999,2017-05-01 12:00:00,0.700424


In [ ]:
data_agg = data.groupby(["longitude", "latitude"]).agg({"v10" : "mean"}).reset_index()
data_agg.head()

,longitude,latitude,v10
0,-73.900002,-7.6,-0.279414
1,-73.900002,-7.5,-0.301488
2,-73.900002,-7.4,-0.303141
3,-73.800003,-7.7,-0.240868
4,-73.800003,-7.6,-0.258600


A parte da filtragem não é necessária se for para todos os municípios do BR.

In [ ]:
points = gpd.read_file("/content/drive/MyDrive/Climate_Data_Files/Br_mun_centroides.shp")

data_agg = data_agg.rename(columns={"longitude" : "lon", "latitude" : "lat"})

data_agg = data_agg.astype(float)

## Filtrando o shape
#point = points.copy()

#points2 = point[(points["SIGLA_UF"] == "MS") | (point["SIGLA_UF"] == "MT") | (point["SIGLA_UF"] == "GO") | (point["SIGLA_UF"] == "DF")].reset_index()


In [ ]:
data_agg['lon'] = data_agg['lon'].round(1)
data_agg['lat'] = data_agg['lat'].round(1)
data_agg.head()

,lon,lat,v10
0,-73.9,-7.6,-0.279414
1,-73.9,-7.5,-0.301488
2,-73.9,-7.4,-0.303141
3,-73.8,-7.7,-0.240868
4,-73.8,-7.6,-0.258600


In [ ]:
points.head()

,index,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,geometry
0,5103,5000203,Água Clara,MS,7781.558,POINT (-52.92669 -19.96282)
1,5104,5000252,Alcinópolis,MS,4397.518,POINT (-53.74919 -18.20915)
2,5105,5000609,Amambai,MS,4193.742,POINT (-54.96817 -23.15781)
3,5106,5000708,Anastácio,MS,2913.177,POINT (-55.77207 -20.69624)
4,5107,5000807,Anaurilândia,MS,3415.657,POINT (-52.80484 -22.11146)


In [ ]:
# Extraindo lat long da coluna geometry

points["lon"] = points.geometry.apply(lambda p: p.x).astype(float)
points["lat"] = points.geometry.apply(lambda p: p.y).astype(float)

points['lon'] = points['lon'].round(1)
points['lat'] = points['lat'].round(1)

In [ ]:
points.head()

,index,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,geometry,lon,lat
0,5103,5000203,Água Clara,MS,7781.558,POINT (-52.92669 -19.96282),-52.9,-20.0
1,5104,5000252,Alcinópolis,MS,4397.518,POINT (-53.74919 -18.20915),-53.7,-18.2
2,5105,5000609,Amambai,MS,4193.742,POINT (-54.96817 -23.15781),-55.0,-23.2
3,5106,5000708,Anastácio,MS,2913.177,POINT (-55.77207 -20.69624),-55.8,-20.7
4,5107,5000807,Anaurilândia,MS,3415.657,POINT (-52.80484 -22.11146),-52.8,-22.1


In [ ]:
points.shape

(467, 8)

In [ ]:
base = pd.merge(points, data_agg, how= "inner", on= ["lat", "lon"])

In [ ]:
base.head()

,index,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,geometry,lon,lat,v10
0,5103,5000203,Água Clara,MS,7781.558,POINT (-52.92669 -19.96282),-52.9,-20.0,-0.587389
1,5104,5000252,Alcinópolis,MS,4397.518,POINT (-53.74919 -18.20915),-53.7,-18.2,-0.512258
2,5105,5000609,Amambai,MS,4193.742,POINT (-54.96817 -23.15781),-55.0,-23.2,-0.850739
3,5106,5000708,Anastácio,MS,2913.177,POINT (-55.77207 -20.69624),-55.8,-20.7,-0.405276
4,5107,5000807,Anaurilândia,MS,3415.657,POINT (-52.80484 -22.11146),-52.8,-22.1,-0.581506


In [ ]:
base.shape

(467, 9)

In [ ]:
base.loc[base['NM_MUN'] == 'Dourados']

,index,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,geometry,lon,lat,v10
31,5134,5003702,Dourados,MS,4062.236,POINT (-54.83888 -22.14486),-54.8,-22.1,-0.460154


In [ ]:
#base["d2m"] = base["d2m"] - 273.15
#base.head()

In [ ]:
base.head()

,index,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,geometry,lon,lat,v10
0,5103,5000203,Água Clara,MS,7781.558,POINT (-52.92669 -19.96282),-52.9,-20.0,-0.587389
1,5104,5000252,Alcinópolis,MS,4397.518,POINT (-53.74919 -18.20915),-53.7,-18.2,-0.512258
2,5105,5000609,Amambai,MS,4193.742,POINT (-54.96817 -23.15781),-55.0,-23.2,-0.850739
3,5106,5000708,Anastácio,MS,2913.177,POINT (-55.77207 -20.69624),-55.8,-20.7,-0.405276
4,5107,5000807,Anaurilândia,MS,3415.657,POINT (-52.80484 -22.11146),-52.8,-22.1,-0.581506


In [ ]:
base.to_csv('/content/drive/MyDrive/Climate_Data_Files/Dataset_Final/v_wind_final2017.csv')